In [6]:
 pip install pandas geopy rapidfuzz

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from geopy.geocoders import Nominatim
from rapidfuzz import process, fuzz
import re
import time


In [8]:
data= pd.read_csv('address_data.csv')

In [9]:
ABBR_DICT = {
    "St": "Street",
    "Rd": "Road",
    "Pkwy": "Parkway",
    "Plc": "Place",
    "Brg": "Bridge",
    "Bzr": "Bazaar"
}

In [10]:
INDIAN_CITIES = [
    "Bengaluru", "Mumbai", "Delhi", "Hyderabad", "Chennai", "Kolkata", "Ahmedabad",
    "Pune", "Jaipur", "Lucknow", "Kanpur", "Nagpur", "Indore", "Thane", "Bhopal",
    "Visakhapatnam", "Patna", "Vadodara", "Ghaziabad", "Ludhiana", "Agra", "Nashik",
    "Faridabad", "Meerut", "Rajkot", "Varanasi", "Srinagar", "Aurangabad", "Dhanbad",
    "Amritsar", "Prayagraj", "Ranchi", "Howrah", "Coimbatore", "Jabalpur", "Gwalior",
    "Vijayawada", "Jodhpur", "Madurai", "Raipur", "Kota", "Guwahati", "Chandigarh",
    "Solapur", "Hubli", "Mysuru", "Tiruchirappalli", "Bareilly", "Aligarh", "Moradabad",
    "Gurgaon", "Noida", "Greater Noida", "Dehradun", "Shimla", "Thiruvananthapuram",
    "Kozhikode", "Ernakulam", "Mangalore", "Warangal", "Nellore", "Tirupati"
]

In [11]:
def expand_abbreviations(addr):
    for abbr, full in ABBR_DICT.items():
        addr = re.sub(r'\b' + re.escape(abbr) + r'\b', full, addr)
    return addr

In [12]:
def normalize_address(addr):
    addr = addr.lower()
    addr = re.sub(r'\s+', ' ', addr)  # collapse multiple spaces
    addr = re.sub(r',+', ',', addr)   # collapse multiple commas
    addr = addr.strip()
    return addr

In [13]:
def correct_city(addr):
    match, score = process.extractOne(addr, INDIAN_CITIES, scorer=fuzz.token_sort_ratio)
    if score > 80:
        # Replace city portion with standardized city
        for city in INDIAN_CITIES:
            if city.lower() in addr:
                addr = addr.replace(city.lower(), match)
        return addr
    return addr

In [14]:
def clean_address(addr):
    addr = expand_abbreviations(addr)
    addr = normalize_address(addr)
    addr = correct_city(addr)
    return addr

In [15]:
data['clean_address'] = data['address'].apply(clean_address)

ValueError: too many values to unpack (expected 2)